# HW2: Airport Terminal Design

#### Authors: Cameron Irmas, Hannah Mankle

### 1

#### Description: With text and code, show how you check if you have achieved the goal.

In order to determine if we have reached the goal, we need to ensure that:
1. The desired number of terminals has been reached
2. No rules are violated anywhere in the matrix

Assuming that we apply the rules as stated at the transition point (children are valid), we can maintain a number of terminals for any given child node, and check against that, avoiding the need for extra computation (though needs to be accounted for in the transition process).

In [2]:
struct AirportNode
    value
    depth::Int
    terminals::Int
end

function is_goal(node::AirportNode, terminals::Int)
    return node.terminals == terminals
end

is_goal (generic function with 1 method)

### 2

#### Description: With text and code, show how you represent the candidates, and transitions for this problem.

Candidates are represented above as `AirportNode`. We can maintain a current state of the airport in a matrix, the current depth, and the number of terminals present at this current state. As mentioned, we can carefully maintain a  current number of terminals by incrementing whenever a new terminal is added with that new child node.

Transitions in a typical DFS are represented as a list of functions representing transformations, resulting in child nodes to be added to the stack. In this case, we have two transition functions, one representing 0->1, and the other representing 0->2. Each function will evaluate the current state of the matrix, and any given point.